# Global Minimial Viable Products (1)

Combine:
* SMAP available porosity
* ERA5 runoff
* MERIT terrain slope
* USDA soil texture class

into global product for PRZ water restoration

In [24]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
from earthshot import water_viz as vis
from earthshot import water_common as common
from earthshot import normalize as norm
from earthshot import mon_stats
from statistics import mean
import folium
from folium import plugins

In [3]:
#ee.Authenticate()
ee.Initialize()

In [4]:
#common parameters
bbox_global = ee.Geometry.BBox(-180,-90,180,90)

# SMAP-USDA Available Porosity
More available porosity is better.

In [13]:
# Sum up available porosity in mm across the year
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = (smap_usda_clim
                  .filter(ee.Filter.eq('band', 'avail_porosity_mm')))

avail_porosity_img = avail_porosity.sum()

avail_porosity_img_scaled = (norm.img_scale(
    avail_porosity_img, area_of_interest= bbox_global))

## ERA5 Runoff

In [46]:
# Sum up runoff across the year to include in product score
runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')

runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)

runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum()

runoff_img_scaled = (norm.img_scale(
    runoff_img, 
    area_of_interest= bbox_global))

runoff_range = norm.img_range(runoff_img_scaled, area_of_interest = bbox_global)

## MERIT Terrain Slope

In [47]:
#include slope as a factor in scoring product, mask for lower slopes
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(10).And(slope_img.gt(0))
slope_img_masked = slope_img.updateMask(slope_mask).pow(-1)
slope_img_scaled = norm.img_scale(slope_img_masked, area_of_interest=bbox_global)

# Soil Types

In [18]:
#incorporate soil types as a factor in scoring for PRZ
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [19]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.7" +
        ": (b('b0') == 7) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 5) ? 0.7" +
        ": (b('b0') == 4) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 2) ? 0.4" +
        ": (b('b0') == 1) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 12) ? 1.0" +
        ": (b('b10') == 11) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 9) ? 1.0" +
        ": (b('b10') == 8) ? 0.7" +
        ": (b('b10') == 7) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 5) ? 0.7" +
        ": (b('b10') == 4) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 2) ? 0.4" +
        ": (b('b10') == 1) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 12) ? 1.0" +
        ": (b('b30') == 11) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 9) ? 1.0" +
        ": (b('b30') == 8) ? 0.7" +
        ": (b('b30') == 7) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 5) ? 0.7" +
        ": (b('b30') == 4) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 2) ? 0.4" +
        ": (b('b30') == 1) ? 0.4" +
        ": 0")

soil_60 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.7" +
        ": (b('b60') == 4) ? 0.7" +
        ": (b('b60') == 6) ? 0.7" +
        ": (b('b60') == 9) ? 0.7" +
        ": (b('b60') == 1) ? 0.4" +
        ": (b('b60') == 3) ? 0.4" +
        ": (b('b60') == 11) ? 0.4" +
        ": (b('b60') == 12) ? 0.4" +
        ": 0")

soil_100 = soil_types.expression(
        "(b('b100') == 5) ? 1.0" +
        ": (b('b100') == 7) ? 1.0" +
        ": (b('b100') == 8) ? 1.0" +
        ": (b('b100') == 10) ? 1.0" +
        ": (b('b100') == 2) ? 0.7" +
        ": (b('b100') == 4) ? 0.7" +
        ": (b('b100') == 6) ? 0.7" +
        ": (b('b100') == 9) ? 0.7" +
        ": (b('b100') == 1) ? 0.4" +
        ": (b('b100') == 3) ? 0.4" +
        ": (b('b100') == 11) ? 0.4" +
        ": (b('b100') == 12) ? 0.4" +
        ": 0")

soil_200 = soil_types.expression(
        "(b('b200') == 5) ? 1.0" +
        ": (b('b200') == 7) ? 1.0" +
        ": (b('b200') == 8) ? 1.0" +
        ": (b('b200') == 10) ? 1.0" +
        ": (b('b200') == 2) ? 0.7" +
        ": (b('b200') == 4) ? 0.7" +
        ": (b('b200') == 6) ? 0.7" +
        ": (b('b200') == 9) ? 0.7" +
        ": (b('b200') == 1) ? 0.4" +
        ": (b('b200') == 3) ? 0.4" +
        ": (b('b200') == 11) ? 0.4" +
        ": (b('b200') == 12) ? 0.4" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

bottom_soils = soil_60.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_60.select('constant'),
                                  'soil_100': soil_100.select('constant'),
                                  'soil_200': soil_200.select('constant')})

In [20]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=bbox_global)
bottom_soils_scaled = norm.img_scale(bottom_soils, area_of_interest=bbox_global)

## PRZ Score
Simply give higher scores to places that have both high runoff, available porosity, favorable soil type and slope. Masked for lower slopes.

In [48]:
score = (runoff_img_scaled.multiply(.25)
         .add(avail_porosity_img_scaled).multiply(.25)
         .add(bottom_soils_scaled.multiply(.25))
         .add(slope_img_scaled.multiply(.25)))
score = norm.img_scale(score, area_of_interest=bbox_global).rename('score')
score_range = norm.img_range(score, area_of_interest=bbox_global)

In [49]:
#visualizing PRZ score

palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum= score_range[0], maximum= score_range[1])

box_corners = bbox_global.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': score_range[0], 'max': score_range[1], 'dimensions': 512,
    'palette': palette}



In [50]:
#creating the map, visualizing score

the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=1, height=400)

the_map.add_ee_layer(score, vis_params, name = 'PRZ Score')

vis.folium_display(the_map)

